# Tutorial: Image Classification using Automated ML

In this tutorial, you use automated machine learning in Azure Machine Learning service to  train an image classification model using the [MNIST](https://azure.microsoft.com/services/open-datasets/catalog/mnist/) dataset. This process accepts training data and configuration settings, and automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model.

You will learn how to:

> * Download a dataset and look at the data
> * Train a machine learning image classification model using autoML 
> * Explore the results


### Connect to your workspace and create an experiment

You start with importing some libraries and creating an experiment to track the runs in your workspace. A workspace can have multiple experiments, and all the users that have access to the workspace can collaborate on them. 

In [ ]:
import logging

from matplotlib import pyplot as plt
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mnist-automl-sdk'

experiment=Experiment(ws, experiment_name)

## Import Data

Before you train a model, you need to understand the data that you are using to train it. In this section you will:

* Download the MNIST dataset
* Display some sample images

### Download the MNIST dataset

Use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Each dataset has a corrseponding class, `MNIST` in this case, to retrieve the data in different ways.

Follow this [how-to](https://aka.ms/azureml/howto/createdatasets) if you want to learn more about Datasets and how to use them.

To decide: should I add visualization of the data like in the other notebooks?



In [ ]:
from azureml.opendatasets import MNIST

mnist_dataset = MNIST.get_tabular_dataset()
mnist_dataset_sample = mnist_dataset.take_sample(0.1,seed=42) # we will only download a small sample for this tutorial 
training_data, validation_data = mnist_dataset_sample.random_split(percentage=0.8, seed=223)
label_column_name = 'label'



## Train



When you use automated machine learning in Azure ML, you input training data and configuration settings, and the process automatically iterates through combinations of different feature normalization/standardization methods, models, and hyperparameter settings to arrive at the best model. 
Learn more about how you configure automated ML [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train).


Instantiate an [AutoMLConfig](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py) object. This defines the settings and data used to run the experiment.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. 
|**enable_early_stopping**  | Stop the run if the metric score is not showing improvement.|
|**n_cross_validations**|Number of cross validation splits.|
|**training_data**|Input dataset, containing both features and label column.|
|**label_column_name**|The name of the label column.|

You can find more information about primary metrics [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train#primary-metric)

In [ ]:
automl_settings = {
    "primary_metric": 'accuracy',
    "experiment_timeout_hours": 0.25, # This is a time limit for testing purposes, remove it for real use cases, this will drastically limit ability to find the best model possible
    "verbosity": logging.INFO,
    "enable_stack_ensemble": False
}

automl_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automl_errors.log',
                             training_data = training_data,
                             label_column_name = label_column_name,
                             **automl_settings
                            )

Call the `submit` method on the experiment object and pass the run configuration. 

**Note: Depending on the data and the number of iterations an AutoML run can take a while to complete.**

In this example, we specify `show_output = True` to print currently running iterations to the console. 


To be decided: what to show, this experience or the widget?

In [ ]:
local_run = experiment.submit(automl_config, show_output = False)
# change to show_output=False

In [ ]:
local_run

#### Widget for Monitoring Runs

The widget will first report a "loading" status while running the first iteration. After completing the first iteration, an auto-updating graph and table will be shown. The widget will refresh once per minute, so you should see the graph update as child runs complete.

**Note:** The widget displays a link at the bottom. Use this link to open a web interface to explore the individual run details

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

### Analyze results

Below we select the best model from our iterations. The `get_output` method on `automl_classifier` returns the best run and the model for the run.

In [ ]:
best_run, best_model = local_run.get_output()
best_model


Now that the model is trained, we'll run the test data through the trained model to get the predicted values.

In [ ]:
# convert the test data to dataframe
X_test_df = validation_data.drop_columns(columns=[label_column_name]).to_pandas_dataframe()
y_test_df = validation_data.keep_columns(columns=[label_column_name], validate=True).to_pandas_dataframe()


In [ ]:
# call the predict functions on the model
y_pred = best_model.predict(X_test_df)
y_pred


### Calculate metrics for the prediction

Now visualize the data to show what our truth (actual) values are compared to the predicted values 
from the trained model that was returned.

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np
import itertools

cf =confusion_matrix(y_test_df.values,y_pred)
plt.imshow(cf,cmap=plt.cm.Blues,interpolation='nearest')
plt.colorbar()
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.xticks(np.arange(10), ('0,','1', '2', '3', '4', '5','6','7','8','9'))
plt.yticks(np.arange(10), ('0,','1', '2', '3', '4', '5','6','7','8','9'))
# plotting text value inside cells
thresh = cf.max() / 2.
for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
    plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] >thresh else 'black')
plt.show()

## Control cost and clean up resources

To control cost and clean up the resources after this quickstart, there are several things you can do.



If you want to control cost you can also stop the compute instance this notebook is running on by following these steps:

1. Go to **Compute** in the left-hand menu of the Azure Machine Learning studio
1. Select your compute instance
1. Select **Stop**


**Important: The resources you created can be used as prerequisites to other Azure Machine Learning tutorials and how-to articles.** If you don't plan to use the resources you created, delete them, so you don't incur any charges:

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

To be decided: Should these instructions really be a part of the tutorials in this experience? What are the consequences of removing them? 